# ClickHouse

> [ClickHouse](https://clickhouse.com/) 是最快、资源效率最高的开源数据库，适用于实时应用程序和分析，提供完整的 SQL 支持以及广泛的函数，可协助用户编写分析性查询。最近添加的数据结构和距离搜索函数（如 `L2Distance`）以及[近似最近邻搜索索引](https://clickhouse.com/docs/en/engines/table-engines/mergetree-family/annindexes) 使 ClickHouse 能够用作高性能、可扩展的向量数据库，以 SQL 的方式存储和搜索向量。

本笔记本展示了如何使用与 `ClickHouse` 向量存储相关的功能。

## 设置

首先使用 docker 设置本地 clickhouse服务器：

In [ ]:
! docker run -d -p 8123:8123 -p9000:9000 --name langchain-clickhouse-server --ulimit nofile=262144:262144 clickhouse/clickhouse-server:24.7.6.8

您需要安装 `langchain-community` 和 `clickhouse-connect` 才能使用此集成

In [ ]:
pip install -qU langchain-community clickhouse-connect

### 凭据

此Notebook无需凭据，只需确保已安装上述软件包。

如果你希望获得一流的自动化模型调用跟踪，也可以通过取消注释以下内容来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

## 实例化

import EmbeddingTabs from "@theme/EmbeddingTabs";

<EmbeddingTabs/>

In [ ]:
# | output: false
# | echo: false
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
from langchain_community.vectorstores import Clickhouse, ClickhouseSettings

settings = ClickhouseSettings(table="clickhouse_example")
vector_store = Clickhouse(embeddings, config=settings)

## 管理向量存储

创建向量存储后，我们可以通过添加和删除不同的项目与其进行交互。

### 向向量存储添加项目

我们可以使用 `add_documents` 函数向向量存储添加项目。

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

### 从向量存储中删除项目

我们可以使用 `delete` 函数通过 ID 从向量存储中删除项目。

In [ ]:
vector_store.delete(ids=uuids[-1])

## 查询向量存储

一旦创建了向量存储并添加了相关文档，您最有可能希望在链或代理运行时查询它。

### 直接查询

#### Similarity search

执行简单的相似性搜索，如下所示：

In [ ]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy", k=2
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

#### 相似性搜索（含得分）

您也可以通过得分进行搜索：

In [ ]:
results = vector_store.similarity_search_with_score("Will it be hot tomorrow?", k=1)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

## 筛选

您可以直接访问 ClickHouse SQL `WHERE` 语句。您可以按照标准 SQL 编写 `WHERE` 子句。

**注意**: 请注意 SQL 注入的风险，此接口不得直接由最终用户调用。

如果您在设置中自定义了 `column_map`，您可以像这样使用筛选器进行搜索：

In [ ]:
meta = vector_store.metadata_column
results = vector_store.similarity_search_with_relevance_scores(
    "What did I eat for breakfast?",
    k=4,
    where_str=f"{meta}.source = 'tweet'",
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

#### 其他搜索方法

本笔记本未涵盖各种其他搜索方法，例如 MMR 搜索或按向量搜索。有关 `Clickhouse` vector store 可用的搜索功能的完整列表，请参阅 [API 参考](https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.clickhouse.Clickhouse.html)。

### 通过转换为检索器进行查询

您也可以将向量存储转换为检索器，以便在链中使用更加便捷。

以下是如何将向量存储转换为检索器，然后使用简单查询和过滤器调用检索器的方法。

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

## 为检索增强生成而使用

有关如何为此向量存储进行检索增强生成 (RAG) 的指南，请参阅以下章节：

- [教程](/docs/tutorials/)
- [操作方法：使用 RAG 进行问答](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [检索概念文档](https://python.langchain.com/docs/concepts/retrieval)

欲了解更多信息，请查看使用 Astra DB 的完整 RAG 模板：[here](https://github.com/langchain-ai/langchain/tree/master/templates/rag-astradb)。

## API 参考

有关所有 `Clickhouse` 功能和配置的详细文档，请访问 API 参考：https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.clickhouse.Clickhouse.html